In [0]:
%sql

----------------- Add sourceoftruth to default.viw_additionalfields ----------------

ALTER VIEW default.viw_additionalfields AS
SELECT DISTINCT
  rd.visualrequestfilenumber AS `Request ID`,
  frccf.identityverification,
  frccf.previousfoirequest,
  frccf.physicalpageestimate,
  frccf.electronicpageestimate,
  frccf.noofpagesdeduplicated,
  frccf.applicationfeepaid,
  frccf.applicationfeerefunded,
  frccf.applicationfeetransferred,
  frccf.linkedrequests,
  eco.officecode AS `Action Office`,
  rd.closeddate AS `Closed Date`,
  rd.targetdate AS `Due Date`,
  rd.foirequestid,
  rd.officeid,
  rd.visualrequestfilenumber,
  rd.amendmentcreateddate,
  rd.refvisualrequestfilenumber,
  rd.closedby,
  rd.closeddate,
  rd.notes,
  rd.receivedbyusername,
  rd.createddate,
  rd.currentactivity,
  rd.deliverydate,
  rd.targetdate,
  rd.disposition,
  rd.originalcloseddate,
  rd.originaltargetdate,
  rd.originalreceiveddate,
  rd.primaryusername,
  rd.processeddays,
  rd.receiveddate,
  rd.referencenumber,
  rd.remainingdays,
  rd.requestdescription,
  rd.caseowner,
  rd.status,
  rd.startdate,
  rd.shipaddressid,
  rd.billaddressid,
  rd.receivedmodeid,
  rd.deliverymodeid,
  rd.primarygroupid,
  rd.requesttypeid,
  rd.onbehalfofrequesterid,
  rd.sourceoftruth
FROM
  factRequestDetails rd
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
WHERE
  rd.activeflag = 'Y'

In [0]:
%sql

----------------- Update LEFT JOIN dimRequesters req to default.viw_sampletlreport ----------------

ALTER VIEW default.viw_sampletlreport AS
SELECT
  rd.visualrequestfilenumber AS Request_ID,
  rt.requesttypename AS Request_Type,
  category.requestertypename AS Applicant_Category,
  rd.redactiondescription AS Request_Description,
  rd.startdate AS Start_Date,
  rd.targetdate AS Due_Date,
  rd.remainingdays AS Remaining_Days,
  rd.currentactivity AS Current_Action,
  rd.requeststatus AS Request_Status,
  rd.primaryusername AS Primary_User,
  rd.execcomments AS Comments,
  rd.extension AS Number_of_Extensions_Applied,
  lastinvoice.invoiceddate AS Last_Invoice_Date,
  rdd.noofpagesintherequest AS Total_pages_in_request,
  eco.officedescription AS Office_Name,
  eco.officecode AS Office_Code,
  rd.caseowner AS Request_Owner,
  CONCAT(req.lastname, ', ', req.firstname) AS Applicant_Name,
  rd.groupname AS User_Group,
  rd.requesteddate AS Requested_Date,
  rd.receiveddate AS Received_Date,
  rd.closeddate AS Closed_Date,
  rd.originalcloseddate AS Original_Closed_Date,
  rd.requestage AS Request_Age
FROM
  factRequestDetails rd
    LEFT JOIN dimRequests r
      ON (
        rd.foirequestid = r.foirequestid
        and rd.sourceoftruth = r.sourceoftruth
      )
    LEFT JOIN dimRequestTypes rt
      ON rt.requesttypeid = rd.requesttypeid
    LEFT JOIN (
      SELECT
        foirequestid,
        invoiceddate,
        ROW_NUMBER() OVER (PARTITION BY foirequestid ORDER BY createddate DESC) AS rn
      FROM
        factRequestInvoices
      WHERE
        activeflag = 'Y'
    ) lastinvoice
      ON lastinvoice.foirequestid = rd.foirequestid
      AND lastinvoice.rn = 1
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rdd.foirequestid = rd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN dimECOffice eco
      ON eco.officeid = rd.officeid
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes category
      ON rr.requestertypeid = category.requestertypeid
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rr.sourceoftruth IS NULL OR rr.sourceoftruth != 'FOIMOD') AND rr.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rr.sourceoftruth = 'FOIMOD' AND req.foirequestid = rr.foirequestid AND req.requesterid = rr.requesterid)
WHERE
  rd.activeflag = 'Y'

In [0]:
%sql

----------------- Update LEFT JOIN dimRequesters req to default.viw_analystworkloaddash

ALTER VIEW default.viw_analystworkloaddash AS
SELECT DISTINCT
  rd.foirequestid,
  rd.visualrequestfilenumber AS requestid,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN 'Intake'
    ELSE ecggn.groupname
  END AS team,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN ecg.groupdescription
    ELSE ecggn.groupdescription
  END AS manager,
  rd.ministry,
  eco.officecode AS procorg,
  rd.applicantcategoryid AS category,
  rt.requesttypename AS type,
  rqt.requestertypename AS applicanttype,
  concat(req.lastname, ', ', req.firstname) AS applicantname,
  frccf.subject,
  rd.startdate,
  rd.closeddate AS enddate,
  rd.targetdate AS duedate,
  rd.processeddays AS totalprocesseddays,
  rd.currentactivity,
  frccf.currentactivitydate,
  rd.primaryusername AS analyst,
  frccf.crossgovtno,
  rd.requestdescription AS description,
  rd.status,
  frccf.customfieldstatus AS `Custom Status`,
  frccf.countontime,
  frccf.countoverdue,
  frccf.onholddays,
  frccf.daysoverdue,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    ELSE 'Not Closed'
  END AS notclosed,
  rpt.estimatedamount AS feesest,
  rpt.incurredamount AS feeswaived,
  rpt.paidamount AS feespaid,
  rd.disposition,
  frccf.publication,
  frccf.publicationreason,
  CASE
    WHEN rd.extension IS NULL THEN 'N'
    ELSE 'Y'
  END AS extension,
  rd.execcomments,
  frccf.noofpagesreleased AS noofpagesdelivered,
  rdd.noofpagesintherequest,
  CASE
    WHEN
      month(rd.receiveddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.receiveddate), year(add_months(rd.receiveddate, 12)))
    WHEN month(rd.receiveddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.receiveddate, -12)), year(rd.receiveddate))
  END AS startfyr,
  CASE
    WHEN
      month(rd.closeddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.closeddate), year(add_months(rd.closeddate, 12)))
    WHEN month(rd.closeddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.closeddate, -12)), year(rd.closeddate))
  END AS endfyr,
  CASE
    WHEN month(rd.receiveddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.receiveddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.receiveddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.receiveddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS startfqtr,
  CASE
    WHEN month(rd.closeddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.closeddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.closeddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.closeddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS endfqtr,
  d.division,
  s.name AS stage,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    WHEN frccf.countontime = 1 THEN 'Active'
    WHEN frccf.countoverdue = 1 THEN 'Overdue'
    ELSE NULL
  END AS overduestatus,
  ffrs.name AS foiflowstatus,
  ffrsd.ministryassigneefullname AS assignedministryperson,
  rds2.divisions,
  rd.requeststatus,
  month(rd.startdate) AS startmonth,
  rd.requesttypeid,
  COALESCE(CAST(ffrsd.recordspagecount AS INT), 0) AS recordspagecount,
  COALESCE(CAST(ffrsd.lanpagecount AS INT), 0) AS lanpagecount
FROM
  factRequestDetails rd
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid
    LEFT JOIN dimECGroups ecggn
      ON rd.groupname = ecggn.groupname
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes rqt
      ON rd.applicantcategoryid = rqt.requestertypeid
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factRequestDivisionalStages
      GROUP BY
        foirequestnumber
    ) urds
      ON urds.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestDivisionalStages rds
      ON rds.foirequestnumber = rd.visualrequestfilenumber
      AND urds.runcycleid = rds.runcycleid
      AND urds.createddate = rds.createddate
    LEFT JOIN dimStages s
      ON s.stageid = rds.stageid
    LEFT JOIN dimDivisions d
      ON d.divisionid = rds.divisionid
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factFOIFlowRequestStatusDetails
      GROUP BY
        foirequestnumber
    ) uffrsd
      ON uffrsd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factFOIFlowRequestStatusDetails ffrsd
      ON ffrsd.foirequestnumber = rd.visualrequestfilenumber
      AND uffrsd.runcycleid = ffrsd.runcycleid
      AND uffrsd.createddate = ffrsd.createddate
    LEFT JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = ffrsd.requeststatusid
    LEFT JOIN (
      SELECT
        rds_1.foirequestnumber,
        string_agg(CONCAT('<p>', d_1.division, ' - ', s_1.name, '</p>'), '\n') AS divisions
      FROM
        (
          SELECT
            foirequestnumber,
            MAX(foirequestversionid) AS foirequestversionid,
            MAX(runcycleid) AS runcycleid
          FROM
            factRequestDivisionalStages
          GROUP BY
            foirequestnumber
        ) sq
          JOIN factRequestDivisionalStages rds_1
            ON rds_1.foirequestnumber = sq.foirequestnumber
            AND rds_1.foirequestversionid = sq.foirequestversionid
            AND rds_1.runcycleid = sq.runcycleid
          JOIN dimStages s_1
            ON s_1.stageid = rds_1.stageid
          JOIN dimDivisions d_1
            ON d_1.divisionid = rds_1.divisionid
      GROUP BY
        rds_1.foirequestnumber
    ) rds2
      ON rds2.foirequestnumber = rd.visualrequestfilenumber
WHERE
  rd.activeflag = 'Y'
  AND rd.officeid NOT IN (1, 10, 21, 33, 41, 45, 49, 51, 55)
  AND rd.requesttypeid IN (31, 33)

In [0]:
%sql

----------------- Update LEFT JOIN dimRequesters req to default.viw_allopenclosed ----------------

ALTER VIEW default.viw_allopenclosed AS
SELECT DISTINCT
  rd.visualrequestfilenumber AS `Request ID`,
  rd.groupname AS `User Group`,
  rd.caseowner AS `Request Owner`,
  rd.ministry AS `Public Body`,
  eco.officecode AS `Office Code`,
  rt.requesttypename AS `Request Type`,
  category.requestertypename AS `Applicant Category`,
  req.fullname AS `Applicant Name`,
  rd.subject AS `Subject`,
  rd.startdate AS `Start Date`,
  rd.targetdate AS `Due Date`,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN rd.closeddate
    ELSE NULL
  END AS `Closed Date`,
  rd.processeddays AS `Total Process Days`,
  rd.currentactivity AS `Current Action`,
  rd.primaryusername AS `Primary User`,
  rd.crossgovtno AS `Cross Gov #`,
  rd.requestdescription AS `Description`,
  rs.requeststatusname AS `Request Status`,
  custom.countontime AS `Count On Time`,
  custom.countoverdue AS `Count Overdue`,
  custom.onholddays AS `On Hold Days`,
  custom.daysoverdue AS `Days Overdue`,
  pt.estimatedamount AS `Fees Est`,
  pt.feewaived AS `Fees Waived`,
  custom.feepaidamount AS `Fees Paid`,
  rd.disposition AS `Disposition`,
  rd.publication AS `Publication`,
  rd.publicationreason AS `Publication Reason`,
  ext.numberofextensions AS `Extension`,
  rdd.noofpagesintherequest AS `Total # of Pages`,
  frd.recordspagecount AS `Total Mod Pages`
FROM
  factRequestDetails rd
    LEFT JOIN dimRequests r
      ON (
        rd.foirequestid = r.foirequestid
        and rd.sourceoftruth = r.sourceoftruth
      )
      AND r.cactive = 'Y'
    LEFT JOIN dimRequestTypes rt
      ON rt.requesttypeid = rd.requesttypeid
      AND rt.cactive = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rdd.foirequestid = rd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN dimECOffice eco
      ON eco.officeid = rd.officeid
    LEFT JOIN dimRequestStatuses rs
      ON rd.requeststatus = rs.requeststatus
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes category
      ON rd.applicantcategoryid = category.requestertypeid
    LEFT JOIN dimRequesters req
      ON
		((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
		OR
		(req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields custom
      ON rd.foirequestid = custom.foirequestid
    LEFT JOIN factRequestInvoices ri
      ON rd.foirequestid = ri.foirequestid
      AND ri.activeflag = 'Y'
    LEFT JOIN dimMODMinistryRequestIDs rid
      ON rd.foirequestid = rid.foirequestid
    LEFT JOIN (
      SELECT
        foirequestid,
        COUNT(DISTINCT requestextid) AS numberofextensions
      FROM
        factRequestExtensions
      WHERE
        approvedstatus = 'A'
        AND activeflag = 'Y'
      GROUP BY
        foirequestid
    ) ext
      ON rd.foirequestid = ext.foirequestid
    LEFT JOIN (
      SELECT DISTINCT
        a.foirequestnumber,
        a.recordspagecount
      FROM
        factFOIFlowRequestStatusDetails a
          JOIN (
            SELECT
              foirequestnumber,
              MAX(version) AS maxv
            FROM
              factFOIFlowRequestStatusDetails
            WHERE
              recordspagecount IS NOT NULL
              AND isactive = TRUE
            GROUP BY
              foirequestnumber
          ) c
            ON c.foirequestnumber = a.foirequestnumber
            AND c.maxv = a.version
      WHERE
        a.recordspagecount IS NOT NULL
        AND a.isactive = TRUE
    ) frd
      ON frd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestPaymentTransaction pt
      ON rd.foirequestid = pt.foirequestid
      AND pt.activeflag = 'Y'
    JOIN (
      SELECT
        MAX(runcycleid) AS maxruncycleid,
        visualrequestfilenumber
      FROM
        factRequestDetails
      WHERE
        activeflag = 'Y'
      GROUP BY
        visualrequestfilenumber
    ) maxruncycle
      ON rd.visualrequestfilenumber = maxruncycle.visualrequestfilenumber
      AND rd.runcycleid = maxruncycle.maxruncycleid
WHERE
  rd.activeflag = 'Y'
  AND (
    rd.closeddate >= TIMESTAMP('2019-04-01 00:00:00')
    OR rd.closeddate = TIMESTAMP('1900-01-01 00:00:00')
  )
  AND eco.officecode NOT IN ('IAO', 'IMB', 'XGR', 'PROAC')

In [0]:
%sql

----------------- Update default.viw_broaderaxisfields ----------------
----------------- LEFT JOIN dimAddress aship
----------------- LEFT JOIN dimAddress abill
----------------- LEFT JOIN dimRequesters rqstr
----------------- LEFT JOIN dimRequesters onbehalfofrequesterid

ALTER VIEW default.viw_broaderaxisfields AS
SELECT
  rd.visualrequestfilenumber AS `Request ID`,
  re.extensiondays AS `# of days extended`,
  frccf.noofdocdelivered AS `# of documents delivered`,
  frccf.noofpagesreleased AS `# of pages released`,
  frccf.noofpagesreviewed AS `# of pages reviewed`,
  eco.officecode AS `Action Office`,
  rd.amendmentcreateddate AS `Amended Date`,
  rpt.estimatedamount AS `Amount Estimated ($)`,
  rpt.paidamount AS `Amount Paid ($)`,
  concat_ws(
    ', ', abill.address1, abill.address2, abill.city, abill.state, abill.country, abill.zipcode
  ) AS `Applicant Billing Address`,
  rqstrt.requestertypename AS `Applicant Category`,
  rqstr.email AS `Applicant Email`,
  rqstr.fax AS `Applicant Fax`,
  rpt.applicationfee AS `Application Fee ($)`,
  rd.refvisualrequestfilenumber AS `Applicant's File Reference`,
  rqstr.home AS `Applicant Home Phone`,
  rqstr.mobile AS `Applicant Mobile`,
  rqstr.fullname AS `Applicant Name`,
  concat_ws(
    ', ', aship.address1, aship.address2, aship.city, aship.state, aship.country, aship.zipcode
  ) AS `Applicant Shipping Address`,
  rqstr.workphone1 AS `Applicant Work Phone1`,
  rqstr.workphone2 AS `Applicant Work Phone2`,
  rpt.invoicedamount - rpt.paidamount AS `Balance Amount ($)`,
  rd.closedby AS `Closed By`,
  rd.closeddate AS `Closed Date`,
  rd.notes AS `Comments`,
  frccf.coordinatednrresponsereqd AS `Coordinated NR Response Req'd`,
  rd.receivedbyusername AS `Created By`,
  rd.createddate AS `Created Date`,
  rd.currentactivity AS `Current Action`,
  rd.deliverydate AS `Delivery Date`,
  dm.deliverymodename AS `Delivery Mode`,
  rd.targetdate AS `Due Date`,
  re.approveddate AS `Extension Approval/Denial Date`,
  re.approvedcomments AS `Extension Approval/Denial Note`,
  re.completeddate AS `Extension Completed Date`,
  re.completedcomments AS `Extension Completion Note`,
  re.comments AS `Extension Note`,
  et.extensiontypename AS `Extension Reason`,
  CASE
    WHEN
      re.cstatus = 'C'
      AND re.approvedstatus = 'A'
    THEN
      'Completed (Approved)'
    WHEN
      re.cstatus = 'P'
      AND re.approvedstatus = 'N'
    THEN
      'Pending'
    WHEN
      re.cstatus = 'A'
      AND re.approvedstatus = 'A'
    THEN
      'Approved'
    WHEN
      re.cstatus = 'D'
      AND re.approvedstatus = 'D'
    THEN
      'Denied'
    WHEN
      re.cstatus = 'C'
      AND re.approvedstatus = 'D'
    THEN
      'Completed (Denied)'
    ELSE NULL
  END AS `Extension Status`,
  rpt.feewaivedescription AS `Fee Waiver Request Reason`,
  rpt.feewaiverequested AS `Fee Waiver Requested`,
  rpt.feewaivegranted AS `Fee Waiver Status`,
  rd.disposition AS `Final Disposition`,
  rpt.invoicedamount AS `Last Invoice Amount ($)`,
  rpt.invoicedate AS `Last Invoice Date`,
  rpt.invoicenumber AS `Last Invoice Number`,
  frccf.applicantfilereference AS `Link Requests`,
  pm.paymentmodename AS `Method of Payment`,
  frccf.noofpagesdeduplicated AS `Number of Pages Deduplicated`,
  onbehalfofrequesterid.fullname AS `On Behalf Of`,
  rqstr.company AS `Organization`,
  rd.originalcloseddate AS `Original Closed Date`,
  rd.originaltargetdate AS `Original Due Date`,
  rd.originalreceiveddate AS `Original Start Date`,
  rpt.paymentreceiveddate AS `Payment Date`,
  rpt.paymentstatus AS `Payment Status`,
  rd.primaryusername AS `Primary User`,
  ecg.groupname AS `Primary User Group`,
  rd.processeddays AS `Processed Days`,
  rd.receiveddate AS `Received Date`,
  rm.receivedmodename AS `Received Mode`,
  rd.referencenumber AS `Reference Number`,
  frccf.refundamount AS `Refund Amount`,
  rd.remainingdays AS `Remaining Days`,
  rd.requestdescription AS `Request Description`,
  rd.caseowner AS `Request Owner`,
  rd.status AS `Request Status`,
  rt.requesttypename AS `Request Type`,
  frccf.secondaryusers AS `Secondary Users`,
  rd.startdate AS `Start Date`,
  frccf.subject AS `Subject`,
  rdd.noofpagesintherequest AS `Total # of pages in a request`,
  rdd.noofpagesinreviewlog AS `Total # of pages in the review log`,
  frccf.onholddays AS `Total Days on Hold`,
  rpt.totalinvoicedamount AS `Total Invoiced ($)`,
  frccf.crossgovtno AS `XGOV`,
  COALESCE(ri.chargedamount, 0) AS `Charged Amount ($)`
FROM
  viw_additionalfields rd
    LEFT JOIN dimRequests r
      ON rd.foirequestid = r.foirequestid
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimAddress aship
      ON
        ((aship.sourceoftruth IS NULL OR aship.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.shipaddressid = aship.addressid)
        OR
        (aship.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND aship.foirequestid = rd.foirequestid)
    LEFT JOIN dimAddress abill
      ON
        ((abill.sourceoftruth IS NULL OR abill.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.shipaddressid = abill.addressid)
        OR
        (abill.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND abill.foirequestid = rd.foirequestid)
    LEFT JOIN factRequestExtensions re
      ON rd.foirequestid = re.foirequestid
      AND re.activeflag = 'Y'
    LEFT JOIN dimExtensionTypes et
      ON re.extensiontypeid = et.extensiontypeid
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesters rqstr
      ON
        ((rqstr.sourceoftruth IS NULL OR rqstr.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rr.requesterid = rqstr.requesterid)
        OR
        (rqstr.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND rqstr.foirequestid = rd.foirequestid AND rqstr.requesterid = rr.requesterid)
    LEFT JOIN dimReceivedModes rm
      ON rd.receivedmodeid = rm.receivedmodeid
    LEFT JOIN dimDeliveryModes dm
      ON rd.deliverymodeid = dm.deliverymodeid
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN dimPaymentModes pm
      ON rpt.paymentmodeid = pm.paymentmodeid
    LEFT JOIN dimRequesters onbehalfofrequesterid
      ON
        ((onbehalfofrequesterid.sourceoftruth IS NULL OR onbehalfofrequesterid.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.onbehalfofrequesterid = onbehalfofrequesterid.requesterid)
        OR
        (onbehalfofrequesterid.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND onbehalfofrequesterid.foirequestid = rd.foirequestid AND onbehalfofrequesterid.requesterid = rd.onbehalfofrequesterid)
    LEFT JOIN dimRequesterTypes rqstrt
      ON rr.requestertypeid = rqstrt.requestertypeid
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
    LEFT JOIN viw_requestinvoices ri
      ON rd.foirequestid = ri.foirequestid

In [0]:
%sql

----------------- Update JOIN dimFOIFlowRequestStatus req to default.viw_avgdaysinrequeststate ----------------
ALTER VIEW default.viw_avgdaysinrequeststate AS
SELECT DISTINCT
  rsd.foiflowrequeststatusdetailsid,
  rsd.runcycleid,
  rsd.foirequestnumber,
  rsd.version,
  rsd.isactive,
  rsd.requeststatusid,
  rsd.ministrycode,
  rsd.assignedgroup,
  rsd.assignedministryperson,
  rsd.assignedministrygroup,
  rsd.startdate,
  rsd.duedate,
  rsd.cfrduedate,
  rsd.closedate,
  rsd.createdby,
  rsd.createddate,
  rsd.modifiedby,
  rsd.modifieddate,
  ffrs.name,
  LAG(ffrs.name) OVER (
      PARTITION BY rsd.foirequestnumber
      ORDER BY rsd.foirequestnumber, rsd.createddate, rsd.requeststatusid
    ) AS laststatusname
FROM
  factFOIFlowRequestStatusDetails rsd
    JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = rsd.requeststatusid and ffrs.sourceoftruth = 'FOIMOD'

In [0]:
%sql

----------------- Update default.viw_analystworkloaddash joins to avoid duplicate rows ----------------
ALTER VIEW default.viw_analystworkloaddash AS

  SELECT DISTINCT
  rd.foirequestid,
  rd.visualrequestfilenumber AS requestid,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN 'Intake'
    ELSE ecggn.groupname
  END AS team,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN ecg.groupdescription
    ELSE ecggn.groupdescription
  END AS manager,
  rd.ministry,
  eco.officecode AS procorg,
  rd.applicantcategoryid AS category,
  rt.requesttypename AS type,
  rqt.requestertypename AS applicanttype,
  concat(req.lastname, ', ', req.firstname) AS applicantname,
  frccf.subject,
  rd.startdate,
  rd.closeddate AS enddate,
  rd.targetdate AS duedate,
  rd.processeddays AS totalprocesseddays,
  rd.currentactivity,
  frccf.currentactivitydate,
  rd.primaryusername AS analyst,
  frccf.crossgovtno,
  rd.requestdescription AS description,
  rd.status,
  frccf.customfieldstatus AS `Custom Status`,
  frccf.countontime,
  frccf.countoverdue,
  frccf.onholddays,
  frccf.daysoverdue,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    ELSE 'Not Closed'
  END AS notclosed,
  rpt.estimatedamount AS feesest,
  rpt.incurredamount AS feeswaived,
  rpt.paidamount AS feespaid,
  rd.disposition,
  frccf.publication,
  frccf.publicationreason,
  CASE
    WHEN rd.extension IS NULL THEN 'N'
    ELSE 'Y'
  END AS extension,
  rd.execcomments,
  frccf.noofpagesreleased AS noofpagesdelivered,
  rdd.noofpagesintherequest,
  CASE
    WHEN
      month(rd.receiveddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.receiveddate), year(add_months(rd.receiveddate, 12)))
    WHEN month(rd.receiveddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.receiveddate, -12)), year(rd.receiveddate))
  END AS startfyr,
  CASE
    WHEN
      month(rd.closeddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.closeddate), year(add_months(rd.closeddate, 12)))
    WHEN month(rd.closeddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.closeddate, -12)), year(rd.closeddate))
  END AS endfyr,
  CASE
    WHEN month(rd.receiveddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.receiveddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.receiveddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.receiveddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS startfqtr,
  CASE
    WHEN month(rd.closeddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.closeddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.closeddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.closeddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS endfqtr,
  d.division,
  s.name AS stage,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    WHEN frccf.countontime = 1 THEN 'Active'
    WHEN frccf.countoverdue = 1 THEN 'Overdue'
    ELSE NULL
  END AS overduestatus,
  ffrs.name AS foiflowstatus,
  ffrsd.ministryassigneefullname AS assignedministryperson,
  rds2.divisions,
  rd.requeststatus,
  month(rd.startdate) AS startmonth,
  rd.requesttypeid,
  COALESCE(CAST(ffrsd.recordspagecount AS INT), 0) AS recordspagecount,
  COALESCE(CAST(ffrsd.lanpagecount AS INT), 0) AS lanpagecount
FROM
  factRequestDetails rd
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid and ecg.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECGroups ecggn
      ON rd.groupname = ecggn.groupname and ecggn.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes rqt
      ON rd.applicantcategoryid = rqt.requestertypeid and rd.sourceoftruth = rqt.sourceoftruth
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid and rd.sourceoftruth = frccf.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factRequestDivisionalStages
      GROUP BY
        foirequestnumber
    ) urds
      ON urds.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestDivisionalStages rds
      ON rds.foirequestnumber = rd.visualrequestfilenumber
      AND urds.runcycleid = rds.runcycleid
      AND urds.createddate = rds.createddate
    LEFT JOIN dimStages s
      ON s.stageid = rds.stageid and s.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimDivisions d
      ON d.divisionid = rds.divisionid and rd.sourceoftruth = d.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factFOIFlowRequestStatusDetails
      GROUP BY
        foirequestnumber
    ) uffrsd
      ON uffrsd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factFOIFlowRequestStatusDetails ffrsd
      ON ffrsd.foirequestnumber = rd.visualrequestfilenumber
      AND uffrsd.runcycleid = ffrsd.runcycleid
      AND uffrsd.createddate = ffrsd.createddate
    LEFT JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = ffrsd.requeststatusid
    LEFT JOIN (
      SELECT
        rds_1.foirequestnumber,
        string_agg(CONCAT('<p>', d_1.division, ' - ', s_1.name, '</p>'), '\n') AS divisions
      FROM
        (
          SELECT
            foirequestnumber,
            MAX(foirequestversionid) AS foirequestversionid,
            MAX(runcycleid) AS runcycleid
          FROM
            factRequestDivisionalStages
          GROUP BY
            foirequestnumber
        ) sq
          JOIN factRequestDivisionalStages rds_1
            ON rds_1.foirequestnumber = sq.foirequestnumber
            AND rds_1.foirequestversionid = sq.foirequestversionid
            AND rds_1.runcycleid = sq.runcycleid
          JOIN dimStages s_1
            ON s_1.stageid = rds_1.stageid
          JOIN dimDivisions d_1
            ON d_1.divisionid = rds_1.divisionid
      GROUP BY
        rds_1.foirequestnumber
    ) rds2
      ON rds2.foirequestnumber = rd.visualrequestfilenumber
WHERE
  rd.activeflag = 'Y'
  AND rd.officeid NOT IN (1, 10, 21, 33, 41, 45, 49, 51, 55)
  AND rd.requesttypeid IN (31, 33)



In [0]:
%sql

----------------- Update default.viw_analystworkloaddash use officeid for FOIMOD requests ----------------
ALTER VIEW default.viw_analystworkloaddash AS

  SELECT DISTINCT
  rd.foirequestid,
  rd.visualrequestfilenumber AS requestid,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN 'Intake'
    ELSE ecggn.groupname
  END AS team,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN ecg.groupdescription
    ELSE ecggn.groupdescription
  END AS manager,
  rd.ministry,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS procorg,
  rd.applicantcategoryid AS category,
  rt.requesttypename AS type,
  rqt.requestertypename AS applicanttype,
  concat(req.lastname, ', ', req.firstname) AS applicantname,
  frccf.subject,
  rd.startdate,
  rd.closeddate AS enddate,
  rd.targetdate AS duedate,
  rd.processeddays AS totalprocesseddays,
  rd.currentactivity,
  frccf.currentactivitydate,
  rd.primaryusername AS analyst,
  frccf.crossgovtno,
  rd.requestdescription AS description,
  rd.status,
  frccf.customfieldstatus AS `Custom Status`,
  frccf.countontime,
  frccf.countoverdue,
  frccf.onholddays,
  frccf.daysoverdue,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    ELSE 'Not Closed'
  END AS notclosed,
  rpt.estimatedamount AS feesest,
  rpt.incurredamount AS feeswaived,
  rpt.paidamount AS feespaid,
  rd.disposition,
  frccf.publication,
  frccf.publicationreason,
  CASE
    WHEN rd.extension IS NULL THEN 'N'
    ELSE 'Y'
  END AS extension,
  rd.execcomments,
  frccf.noofpagesreleased AS noofpagesdelivered,
  rdd.noofpagesintherequest,
  CASE
    WHEN
      month(rd.receiveddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.receiveddate), year(add_months(rd.receiveddate, 12)))
    WHEN month(rd.receiveddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.receiveddate, -12)), year(rd.receiveddate))
  END AS startfyr,
  CASE
    WHEN
      month(rd.closeddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.closeddate), year(add_months(rd.closeddate, 12)))
    WHEN month(rd.closeddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.closeddate, -12)), year(rd.closeddate))
  END AS endfyr,
  CASE
    WHEN month(rd.receiveddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.receiveddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.receiveddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.receiveddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS startfqtr,
  CASE
    WHEN month(rd.closeddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.closeddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.closeddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.closeddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS endfqtr,
  d.division,
  s.name AS stage,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    WHEN frccf.countontime = 1 THEN 'Active'
    WHEN frccf.countoverdue = 1 THEN 'Overdue'
    ELSE NULL
  END AS overduestatus,
  ffrs.name AS foiflowstatus,
  ffrsd.ministryassigneefullname AS assignedministryperson,
  rds2.divisions,
  rd.requeststatus,
  month(rd.startdate) AS startmonth,
  rd.requesttypeid,
  COALESCE(CAST(ffrsd.recordspagecount AS INT), 0) AS recordspagecount,
  COALESCE(CAST(ffrsd.lanpagecount AS INT), 0) AS lanpagecount
FROM
  factRequestDetails rd
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid and ecg.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECGroups ecggn
      ON rd.groupname = ecggn.groupname and ecggn.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes rqt
      ON rd.applicantcategoryid = rqt.requestertypeid and rd.sourceoftruth = rqt.sourceoftruth
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid and rd.sourceoftruth = frccf.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factRequestDivisionalStages
      GROUP BY
        foirequestnumber
    ) urds
      ON urds.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestDivisionalStages rds
      ON rds.foirequestnumber = rd.visualrequestfilenumber
      AND urds.runcycleid = rds.runcycleid
      AND urds.createddate = rds.createddate
    LEFT JOIN dimStages s
      ON s.stageid = rds.stageid and s.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimDivisions d
      ON d.divisionid = rds.divisionid and rd.sourceoftruth = d.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factFOIFlowRequestStatusDetails
      GROUP BY
        foirequestnumber
    ) uffrsd
      ON uffrsd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factFOIFlowRequestStatusDetails ffrsd
      ON ffrsd.foirequestnumber = rd.visualrequestfilenumber
      AND uffrsd.runcycleid = ffrsd.runcycleid
      AND uffrsd.createddate = ffrsd.createddate
    LEFT JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = ffrsd.requeststatusid
    LEFT JOIN (
      SELECT
        rds_1.foirequestnumber,
        string_agg(CONCAT('<p>', d_1.division, ' - ', s_1.name, '</p>'), '\n') AS divisions
      FROM
        (
          SELECT
            foirequestnumber,
            MAX(foirequestversionid) AS foirequestversionid,
            MAX(runcycleid) AS runcycleid
          FROM
            factRequestDivisionalStages
          GROUP BY
            foirequestnumber
        ) sq
          JOIN factRequestDivisionalStages rds_1
            ON rds_1.foirequestnumber = sq.foirequestnumber
            AND rds_1.foirequestversionid = sq.foirequestversionid
            AND rds_1.runcycleid = sq.runcycleid
          JOIN dimStages s_1
            ON s_1.stageid = rds_1.stageid
          JOIN dimDivisions d_1
            ON d_1.divisionid = rds_1.divisionid
      GROUP BY
        rds_1.foirequestnumber
    ) rds2
      ON rds2.foirequestnumber = rd.visualrequestfilenumber
WHERE
  rd.activeflag = 'Y'
  AND rd.officeid NOT IN ("1", "10", "21", "33", "41", "45", "49", "51", "55", 'CLB', 'IIO', 'MGC', 'OBC', 'TIC')
  AND rd.requesttypeid IN (31, 33)



In [0]:
%sql

----------------- use officeid for FOIMOD requests ----------------

ALTER VIEW default.viw_allopenclosed AS
SELECT DISTINCT
  rd.visualrequestfilenumber AS `Request ID`,
  rd.groupname AS `User Group`,
  rd.caseowner AS `Request Owner`,
  rd.ministry AS `Public Body`,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS `Office Code`,
  rt.requesttypename AS `Request Type`,
  category.requestertypename AS `Applicant Category`,
  req.fullname AS `Applicant Name`,
  rd.subject AS `Subject`,
  rd.startdate AS `Start Date`,
  rd.targetdate AS `Due Date`,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN rd.closeddate
    ELSE NULL
  END AS `Closed Date`,
  rd.processeddays AS `Total Process Days`,
  rd.currentactivity AS `Current Action`,
  rd.primaryusername AS `Primary User`,
  rd.crossgovtno AS `Cross Gov #`,
  rd.requestdescription AS `Description`,
  rs.requeststatusname AS `Request Status`,
  custom.countontime AS `Count On Time`,
  custom.countoverdue AS `Count Overdue`,
  custom.onholddays AS `On Hold Days`,
  custom.daysoverdue AS `Days Overdue`,
  pt.estimatedamount AS `Fees Est`,
  pt.feewaived AS `Fees Waived`,
  custom.feepaidamount AS `Fees Paid`,
  rd.disposition AS `Disposition`,
  rd.publication AS `Publication`,
  rd.publicationreason AS `Publication Reason`,
  ext.numberofextensions AS `Extension`,
  rdd.noofpagesintherequest AS `Total # of Pages`,
  frd.recordspagecount AS `Total Mod Pages`
FROM
  factRequestDetails rd
    LEFT JOIN dimRequests r
      ON (
        rd.foirequestid = r.foirequestid
        and rd.sourceoftruth = r.sourceoftruth
      )
      AND r.cactive = 'Y'
    LEFT JOIN dimRequestTypes rt
      ON rt.requesttypeid = rd.requesttypeid
      AND rt.cactive = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rdd.foirequestid = rd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN dimECOffice eco
      ON eco.officeid = rd.officeid
    LEFT JOIN dimRequestStatuses rs
      ON rd.requeststatus = rs.requeststatus
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes category
      ON rd.applicantcategoryid = category.requestertypeid
    LEFT JOIN dimRequesters req
      ON
		((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
		OR
		(req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields custom
      ON rd.foirequestid = custom.foirequestid
    LEFT JOIN factRequestInvoices ri
      ON rd.foirequestid = ri.foirequestid
      AND ri.activeflag = 'Y'
    LEFT JOIN dimMODMinistryRequestIDs rid
      ON rd.foirequestid = rid.foirequestid
    LEFT JOIN (
      SELECT
        foirequestid,
        COUNT(DISTINCT requestextid) AS numberofextensions
      FROM
        factRequestExtensions
      WHERE
        approvedstatus = 'A'
        AND activeflag = 'Y'
      GROUP BY
        foirequestid
    ) ext
      ON rd.foirequestid = ext.foirequestid
    LEFT JOIN (
      SELECT DISTINCT
        a.foirequestnumber,
        a.recordspagecount
      FROM
        factFOIFlowRequestStatusDetails a
          JOIN (
            SELECT
              foirequestnumber,
              MAX(version) AS maxv
            FROM
              factFOIFlowRequestStatusDetails
            WHERE
              recordspagecount IS NOT NULL
              AND isactive = TRUE
            GROUP BY
              foirequestnumber
          ) c
            ON c.foirequestnumber = a.foirequestnumber
            AND c.maxv = a.version
      WHERE
        a.recordspagecount IS NOT NULL
        AND a.isactive = TRUE
    ) frd
      ON frd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestPaymentTransaction pt
      ON rd.foirequestid = pt.foirequestid
      AND pt.activeflag = 'Y'
    JOIN (
      SELECT
        MAX(runcycleid) AS maxruncycleid,
        visualrequestfilenumber
      FROM
        factRequestDetails
      WHERE
        activeflag = 'Y'
      GROUP BY
        visualrequestfilenumber
    ) maxruncycle
      ON rd.visualrequestfilenumber = maxruncycle.visualrequestfilenumber
      AND rd.runcycleid = maxruncycle.maxruncycleid
WHERE
  rd.activeflag = 'Y'
  AND (
    rd.closeddate >= TIMESTAMP('2019-04-01 00:00:00')
    OR rd.closeddate = TIMESTAMP('1900-01-01 00:00:00')
  )
  AND eco.officecode NOT IN ('IAO', 'IMB', 'XGR', 'PROAC')

In [0]:
%sql

----------------- Use officeid for FOIMOD requests ----------------

ALTER VIEW default.viw_additionalfields AS
SELECT DISTINCT
  rd.visualrequestfilenumber AS `Request ID`,
  frccf.identityverification,
  frccf.previousfoirequest,
  frccf.physicalpageestimate,
  frccf.electronicpageestimate,
  frccf.noofpagesdeduplicated,
  frccf.applicationfeepaid,
  frccf.applicationfeerefunded,
  frccf.applicationfeetransferred,
  frccf.linkedrequests,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS `Action Office`,
  rd.closeddate AS `Closed Date`,
  rd.targetdate AS `Due Date`,
  rd.foirequestid,
  rd.officeid,
  rd.visualrequestfilenumber,
  rd.amendmentcreateddate,
  rd.refvisualrequestfilenumber,
  rd.closedby,
  rd.closeddate,
  rd.notes,
  rd.receivedbyusername,
  rd.createddate,
  rd.currentactivity,
  rd.deliverydate,
  rd.targetdate,
  rd.disposition,
  rd.originalcloseddate,
  rd.originaltargetdate,
  rd.originalreceiveddate,
  rd.primaryusername,
  rd.processeddays,
  rd.receiveddate,
  rd.referencenumber,
  rd.remainingdays,
  rd.requestdescription,
  rd.caseowner,
  rd.status,
  rd.startdate,
  rd.shipaddressid,
  rd.billaddressid,
  rd.receivedmodeid,
  rd.deliverymodeid,
  rd.primarygroupid,
  rd.requesttypeid,
  rd.onbehalfofrequesterid,
  rd.sourceoftruth
FROM
  factRequestDetails rd
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
WHERE
  rd.activeflag = 'Y'

In [0]:
%sql

----------------- Use officeid for FOIMOD requests ----------------

ALTER VIEW default.viw_sampletlreport AS
SELECT
  rd.visualrequestfilenumber AS Request_ID,
  rt.requesttypename AS Request_Type,
  category.requestertypename AS Applicant_Category,
  rd.redactiondescription AS Request_Description,
  rd.startdate AS Start_Date,
  rd.targetdate AS Due_Date,
  rd.remainingdays AS Remaining_Days,
  rd.currentactivity AS Current_Action,
  rd.requeststatus AS Request_Status,
  rd.primaryusername AS Primary_User,
  rd.execcomments AS Comments,
  rd.extension AS Number_of_Extensions_Applied,
  lastinvoice.invoiceddate AS Last_Invoice_Date,
  rdd.noofpagesintherequest AS Total_pages_in_request,
  eco.officedescription AS Office_Name,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS Office_Code,
  rd.caseowner AS Request_Owner,
  CONCAT(req.lastname, ', ', req.firstname) AS Applicant_Name,
  rd.groupname AS User_Group,
  rd.requesteddate AS Requested_Date,
  rd.receiveddate AS Received_Date,
  rd.closeddate AS Closed_Date,
  rd.originalcloseddate AS Original_Closed_Date,
  rd.requestage AS Request_Age
FROM
  factRequestDetails rd
    LEFT JOIN dimRequests r
      ON (
        rd.foirequestid = r.foirequestid
        and rd.sourceoftruth = r.sourceoftruth
      )
    LEFT JOIN dimRequestTypes rt
      ON rt.requesttypeid = rd.requesttypeid
    LEFT JOIN (
      SELECT
        foirequestid,
        invoiceddate,
        ROW_NUMBER() OVER (PARTITION BY foirequestid ORDER BY createddate DESC) AS rn
      FROM
        factRequestInvoices
      WHERE
        activeflag = 'Y'
    ) lastinvoice
      ON lastinvoice.foirequestid = rd.foirequestid
      AND lastinvoice.rn = 1
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rdd.foirequestid = rd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN dimECOffice eco
      ON eco.officeid = rd.officeid
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes category
      ON rr.requestertypeid = category.requestertypeid
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rr.sourceoftruth IS NULL OR rr.sourceoftruth != 'FOIMOD') AND rr.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rr.sourceoftruth = 'FOIMOD' AND req.foirequestid = rr.foirequestid AND req.requesterid = rr.requesterid)
WHERE
  rd.activeflag = 'Y'

In [0]:
%sql

-----------------Use officeid for FOIMOD requests ----------------

ALTER VIEW default.viw_broaderaxisfields AS
SELECT
  rd.visualrequestfilenumber AS `Request ID`,
  re.extensiondays AS `# of days extended`,
  frccf.noofdocdelivered AS `# of documents delivered`,
  frccf.noofpagesreleased AS `# of pages released`,
  frccf.noofpagesreviewed AS `# of pages reviewed`,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS `Action Office`,
  rd.amendmentcreateddate AS `Amended Date`,
  rpt.estimatedamount AS `Amount Estimated ($)`,
  rpt.paidamount AS `Amount Paid ($)`,
  concat_ws(
    ', ', abill.address1, abill.address2, abill.city, abill.state, abill.country, abill.zipcode
  ) AS `Applicant Billing Address`,
  rqstrt.requestertypename AS `Applicant Category`,
  rqstr.email AS `Applicant Email`,
  rqstr.fax AS `Applicant Fax`,
  rpt.applicationfee AS `Application Fee ($)`,
  rd.refvisualrequestfilenumber AS `Applicant's File Reference`,
  rqstr.home AS `Applicant Home Phone`,
  rqstr.mobile AS `Applicant Mobile`,
  rqstr.fullname AS `Applicant Name`,
  concat_ws(
    ', ', aship.address1, aship.address2, aship.city, aship.state, aship.country, aship.zipcode
  ) AS `Applicant Shipping Address`,
  rqstr.workphone1 AS `Applicant Work Phone1`,
  rqstr.workphone2 AS `Applicant Work Phone2`,
  rpt.invoicedamount - rpt.paidamount AS `Balance Amount ($)`,
  rd.closedby AS `Closed By`,
  rd.closeddate AS `Closed Date`,
  rd.notes AS `Comments`,
  frccf.coordinatednrresponsereqd AS `Coordinated NR Response Req'd`,
  rd.receivedbyusername AS `Created By`,
  rd.createddate AS `Created Date`,
  rd.currentactivity AS `Current Action`,
  rd.deliverydate AS `Delivery Date`,
  dm.deliverymodename AS `Delivery Mode`,
  rd.targetdate AS `Due Date`,
  re.approveddate AS `Extension Approval/Denial Date`,
  re.approvedcomments AS `Extension Approval/Denial Note`,
  re.completeddate AS `Extension Completed Date`,
  re.completedcomments AS `Extension Completion Note`,
  re.comments AS `Extension Note`,
  et.extensiontypename AS `Extension Reason`,
  CASE
    WHEN
      re.cstatus = 'C'
      AND re.approvedstatus = 'A'
    THEN
      'Completed (Approved)'
    WHEN
      re.cstatus = 'P'
      AND re.approvedstatus = 'N'
    THEN
      'Pending'
    WHEN
      re.cstatus = 'A'
      AND re.approvedstatus = 'A'
    THEN
      'Approved'
    WHEN
      re.cstatus = 'D'
      AND re.approvedstatus = 'D'
    THEN
      'Denied'
    WHEN
      re.cstatus = 'C'
      AND re.approvedstatus = 'D'
    THEN
      'Completed (Denied)'
    ELSE NULL
  END AS `Extension Status`,
  rpt.feewaivedescription AS `Fee Waiver Request Reason`,
  rpt.feewaiverequested AS `Fee Waiver Requested`,
  rpt.feewaivegranted AS `Fee Waiver Status`,
  rd.disposition AS `Final Disposition`,
  rpt.invoicedamount AS `Last Invoice Amount ($)`,
  rpt.invoicedate AS `Last Invoice Date`,
  rpt.invoicenumber AS `Last Invoice Number`,
  frccf.applicantfilereference AS `Link Requests`,
  pm.paymentmodename AS `Method of Payment`,
  frccf.noofpagesdeduplicated AS `Number of Pages Deduplicated`,
  onbehalfofrequesterid.fullname AS `On Behalf Of`,
  rqstr.company AS `Organization`,
  rd.originalcloseddate AS `Original Closed Date`,
  rd.originaltargetdate AS `Original Due Date`,
  rd.originalreceiveddate AS `Original Start Date`,
  rpt.paymentreceiveddate AS `Payment Date`,
  rpt.paymentstatus AS `Payment Status`,
  rd.primaryusername AS `Primary User`,
  ecg.groupname AS `Primary User Group`,
  rd.processeddays AS `Processed Days`,
  rd.receiveddate AS `Received Date`,
  rm.receivedmodename AS `Received Mode`,
  rd.referencenumber AS `Reference Number`,
  frccf.refundamount AS `Refund Amount`,
  rd.remainingdays AS `Remaining Days`,
  rd.requestdescription AS `Request Description`,
  rd.caseowner AS `Request Owner`,
  rd.status AS `Request Status`,
  rt.requesttypename AS `Request Type`,
  frccf.secondaryusers AS `Secondary Users`,
  rd.startdate AS `Start Date`,
  frccf.subject AS `Subject`,
  rdd.noofpagesintherequest AS `Total # of pages in a request`,
  rdd.noofpagesinreviewlog AS `Total # of pages in the review log`,
  frccf.onholddays AS `Total Days on Hold`,
  rpt.totalinvoicedamount AS `Total Invoiced ($)`,
  frccf.crossgovtno AS `XGOV`,
  COALESCE(ri.chargedamount, 0) AS `Charged Amount ($)`
FROM
  viw_additionalfields rd
    LEFT JOIN dimRequests r
      ON rd.foirequestid = r.foirequestid
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimAddress aship
      ON
        ((aship.sourceoftruth IS NULL OR aship.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.shipaddressid = aship.addressid)
        OR
        (aship.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND aship.foirequestid = rd.foirequestid)
    LEFT JOIN dimAddress abill
      ON
        ((abill.sourceoftruth IS NULL OR abill.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.shipaddressid = abill.addressid)
        OR
        (abill.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND abill.foirequestid = rd.foirequestid)
    LEFT JOIN factRequestExtensions re
      ON rd.foirequestid = re.foirequestid
      AND re.activeflag = 'Y'
    LEFT JOIN dimExtensionTypes et
      ON re.extensiontypeid = et.extensiontypeid
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesters rqstr
      ON
        ((rqstr.sourceoftruth IS NULL OR rqstr.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rr.requesterid = rqstr.requesterid)
        OR
        (rqstr.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND rqstr.foirequestid = rd.foirequestid AND rqstr.requesterid = rr.requesterid)
    LEFT JOIN dimReceivedModes rm
      ON rd.receivedmodeid = rm.receivedmodeid
    LEFT JOIN dimDeliveryModes dm
      ON rd.deliverymodeid = dm.deliverymodeid
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN dimPaymentModes pm
      ON rpt.paymentmodeid = pm.paymentmodeid
    LEFT JOIN dimRequesters onbehalfofrequesterid
      ON
        ((onbehalfofrequesterid.sourceoftruth IS NULL OR onbehalfofrequesterid.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.onbehalfofrequesterid = onbehalfofrequesterid.requesterid)
        OR
        (onbehalfofrequesterid.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND onbehalfofrequesterid.foirequestid = rd.foirequestid AND onbehalfofrequesterid.requesterid = rd.onbehalfofrequesterid)
    LEFT JOIN dimRequesterTypes rqstrt
      ON rr.requestertypeid = rqstrt.requestertypeid
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
    LEFT JOIN viw_requestinvoices ri
      ON rd.foirequestid = ri.foirequestid

In [0]:
%sql

-- get data from materialized view
ALTER VIEW default.viw_allopenclosed AS
SELECT
    visualrequestfilenumber AS `Request ID`,
    groupname AS `User Group`,
    caseowner AS `Request Owner`,
    ministry AS `Public Body`,
    officecode AS `Office Code`,
    requesttypename AS `Request Type`,
    requestertypename AS `Applicant Category`,
    fullname AS `Applicant Name`,
    subject AS `Subject`,
    startdate AS `Start Date`,
    targetdate AS `Due Date`,
    closeddate AS `Closed Date`,
    processeddays AS `Total Process Days`,
    currentactivity AS `Current Action`,
    primaryusername AS `Primary User`,
    crossgovtno AS `Cross Gov #`,
    requestdescription AS `Description`,
    requeststatusname AS `Request Status`,
    countontime AS `Count On Time`,
    countoverdue AS `Count Overdue`,
    onholddays AS `On Hold Days`,
    daysoverdue AS `Days Overdue`,
    estimatedamount AS `Fees Est`,
    feewaived AS `Fees Waived`,
    feepaidamount AS `Fees Paid`,
    disposition AS `Disposition`,
    publication AS `Publication`,
    publicationreason AS `Publication Reason`,
    numberofextensions AS `Extension`,
    noofpagesintherequest AS `Total # of Pages`,
    recordspagecount AS `Total Mod Pages`
FROM default.materialized_viw_allopenclosed;

In [0]:
%sql

-- get data from materialized views
ALTER VIEW default.viw_analystworkloaddash AS

  SELECT
  foirequestid,
  requestid,
  team,
  manager,
  ministry,
  procorg,
  category,
  type,
  applicanttype,
  applicantname,
  subject,
  startdate,
  enddate,
  duedate,
  totalprocesseddays,
  currentactivity,
  currentactivitydate,
  analyst,
  crossgovtno,
  description,
  status,
  CustomStatus AS `Custom Status`,
  countontime,
  countoverdue,
  onholddays,
  daysoverdue,
  notclosed,
  feesest,
  feeswaived,
  feespaid,
  disposition,
  publication,
  publicationreason,
  extension,
  execcomments,
  noofpagesdelivered,
  noofpagesintherequest,
  startfyr,
  endfyr,
  startfqtr,
  endfqtr,
  division,
  stage,
  overduestatus,
  foiflowstatus,
  assignedministryperson,
  divisions,
  requeststatus,
  startmonth,
  requesttypeid,
  recordspagecount,
  lanpagecount
FROM
  default.materialized_viw_analystworkloaddash;

